In [1]:
from nltk.corpus import conll2000
chunked_sent = conll2000.chunked_sents()[0]
print(chunked_sent)

(S
  (NP Confidence/NN)
  (PP in/IN)
  (NP the/DT pound/NN)
  (VP is/VBZ widely/RB expected/VBN to/TO take/VB)
  (NP another/DT sharp/JJ dive/NN)
  if/IN
  (NP trade/NN figures/NNS)
  (PP for/IN)
  (NP September/NNP)
  ,/,
  due/JJ
  (PP for/IN)
  (NP release/NN)
  (NP tomorrow/NN)
  ,/,
  (VP fail/VB to/TO show/VB)
  (NP a/DT substantial/JJ improvement/NN)
  (PP from/IN)
  (NP July/NNP and/CC August/NNP)
  (NP 's/POS near-record/JJ deficits/NNS)
  ./.)


We r using conll2000 corpus to evaluate and train our chunking model. conll2000 corpus contains text data from Wall street journal which r labeled with Part of speech,noun phrase, verb phrase and preposition phrase tags. 
U can see above in the output, data is in the tree format and below data is in iob format.

In [2]:
from nltk.chunk import tree2conlltags, conlltags2tree
iob_tagged = tree2conlltags(chunked_sent)
print(iob_tagged)

[('Confidence', 'NN', 'B-NP'), ('in', 'IN', 'B-PP'), ('the', 'DT', 'B-NP'), ('pound', 'NN', 'I-NP'), ('is', 'VBZ', 'B-VP'), ('widely', 'RB', 'I-VP'), ('expected', 'VBN', 'I-VP'), ('to', 'TO', 'I-VP'), ('take', 'VB', 'I-VP'), ('another', 'DT', 'B-NP'), ('sharp', 'JJ', 'I-NP'), ('dive', 'NN', 'I-NP'), ('if', 'IN', 'O'), ('trade', 'NN', 'B-NP'), ('figures', 'NNS', 'I-NP'), ('for', 'IN', 'B-PP'), ('September', 'NNP', 'B-NP'), (',', ',', 'O'), ('due', 'JJ', 'O'), ('for', 'IN', 'B-PP'), ('release', 'NN', 'B-NP'), ('tomorrow', 'NN', 'B-NP'), (',', ',', 'O'), ('fail', 'VB', 'B-VP'), ('to', 'TO', 'I-VP'), ('show', 'VB', 'I-VP'), ('a', 'DT', 'B-NP'), ('substantial', 'JJ', 'I-NP'), ('improvement', 'NN', 'I-NP'), ('from', 'IN', 'B-PP'), ('July', 'NNP', 'B-NP'), ('and', 'CC', 'I-NP'), ('August', 'NNP', 'I-NP'), ("'s", 'POS', 'B-NP'), ('near-record', 'JJ', 'I-NP'), ('deficits', 'NNS', 'I-NP'), ('.', '.', 'O')]


In [3]:
print(len(conll2000.chunked_words()))
print(len(conll2000.chunked_sents()))

166433
10948


Basic concept of Trigram based chunking model is chunk is predicted based of specific sequence of 3 parts of speech tags in a row. Let's forst divide data into train and test sets. 90% data for training and remaining for testing.

In [6]:
import random
from sklearn.model_selection import train_test_split
shuffled_conll_sents = list(conll2000.chunked_sents())
train_set, test_set = train_test_split(shuffled_conll_sents, test_size=0.1)

In [7]:
# Now Let's create a chunking model. 
from nltk import ChunkParserI, TrigramTagger

In [8]:
class TrigramChunkParser(ChunkParserI):
    def __init__(self, train_set):
 # now will extract only part of speech tags and iob chunked tag pairs.
        train_data = [[(pos_tag, chunk_tag) for word, pos_tag, chunk_tag in tree2conlltags(sent)]
                      for sent in train_set]
 # next will train the TrigramTagger
        self.tagger = TrigramTagger(train_data)
    def parse(self, sentence):
        pos_tags = [pos for word, pos in sentence]
 # now will get the chunked tags
        tagged_pos_tags = self.tagger.tag(pos_tags)
 # then will assemble the word, part of speech and chunk triplets
        conlltags = [(word, pos_tag, chunk_tag)
                    for ((word, pos_tag), (pos_tag, chunk_tag)) in zip(sentence, tagged_pos_tags)]
 # finally will transform the tags into a chunk tree format for our output.
        return conlltags2tree(conlltags)
 # Now we r done building our model. So let's test it out.

In [9]:
# first will train it by passing the training set with TrigramChunkParser
trigram_chunker = TrigramChunkParser(train_set)

In [10]:
print(trigram_chunker.evaluate(test_set))

ChunkParse score:
    IOB Accuracy:  87.0%%
    Precision:     79.5%%
    Recall:        83.1%%
    F-Measure:     81.3%%


we can see that peformance is not bad for very simple chunk parser. N-gram chunk parsers which we just created are a good start but better performance can b achieved using additional features beyond part of speech tags. A popular high performance example is a classification based model. 
As u can see the chunking corpus included with NLTK is valuable resource for training specific chunking models.